In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern, hog
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.image import img_to_array, load_img


from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import cv2
from PIL import Image
from tqdm import tqdm

from skimage.io import imread
from skimage.transform import rescale
from skimage.feature import hog
from skimage import exposure
from skimage import color

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from keras.models import Sequential, Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam, Adamax
from keras.layers import Input, UpSampling2D, Conv2D, concatenate, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import load_img, img_to_array
from keras.regularizers import l1, l2

from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input # Changed import path

import os
from math import ceil
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


Create a Model


In [3]:
def compute_hog(image):
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    block_norm = 'L2-Hys'
    orientations = 9
    stride = 4 
    hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, block_norm=block_norm, visualize=False,
                       feature_vector=True)
    return hog_features

#pixels_per_cell=(16, 16), cells_per_block=(4, 4), orientations=6
# # best parameters found by Bayesian optimization
# def compute_hog(image):
#     pixels_per_cell = (7, 7)
#     cells_per_block = (2, 2)
#     stride = 4 
#     block_norm = 'L2-Hys'
#     orientations = 9
    
    
#     hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
#                        cells_per_block=cells_per_block, block_norm=block_norm, visualize=False,
#                        feature_vector=True)
    
#     return hog_features



In [4]:
# Constants for LBP
LBP_RADIUS = 2
LBP_N_POINTS = 16



def compute_lbp(image):
    lbp = local_binary_pattern(image, P=LBP_N_POINTS, R=LBP_RADIUS, method='default')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 2**LBP_N_POINTS + 1), range=(0, 2**LBP_N_POINTS))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

In [5]:

def compute_sift(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is not None:
        descriptors = descriptors.flatten()[:128]
        if descriptors.shape[0] < 128:
            descriptors = np.pad(descriptors, (0, 128 - descriptors.shape[0]), 'constant')
    else:
        descriptors = np.zeros(128)
    return descriptors

**Try feat_cnn_model with tuned hog parameters on classfication**

In [9]:
# def create_cnn_feature_extractor(cnn_model):
#     # Extract features from the 'conv_7b_ac' layer
#     feature_model = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer('conv_7b_ac').output)
#     return feature_model
# base_model = load_model('/Users/nemekhbayarnomin/Documents/Graduation_project/dataset/3_dataset/eye_diseases_classification/Tuned_InceptionResntV2_full_fundus__model.h5')


# model = create_cnn_feature_extractor(base_model)


# model.summary()

from tensorflow.keras.models import Model

# Modify the CNN model for feature extraction (using the layer before the final Dense layer)
def create_cnn_feature_extractor(cnn_model):
    # Extract features from the Flatten layer
    feature_model = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
    return feature_model


base_model = load_model('feat_cnn_model_softmax_rois_256.h5')
model = create_cnn_feature_extractor(base_model)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_204 (Batch  (None, 111, 111, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                                           

In [12]:
def process_and_extract_features_from_folders(image_folder):
    hog_features_list = []
    lbp_features_list = []
    cnn_features_list = []
    sift_features_list = []
    labels = []

    for label_folder in os.listdir(image_folder):
        folder_path = os.path.join(image_folder, label_folder)
        if not os.path.isdir(folder_path):
            continue

        for image_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, image_name)

            # Read the image
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))
            
                # Prepare image for CNN feature extraction
                img_array = img_to_array(img)
                img_array = img_array / 255.0
                img_array = np.expand_dims(img_array, axis=0)

                # Extract CNN features using the modified model
                cnn_features = model.predict(img_array).flatten()  # Output is already flat
                cnn_features_list.append(cnn_features)

                # Process for HOG, LBP, and SIFT
                img_resized = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


                hog_features_list.append(compute_hog(img_resized).flatten())
                lbp_features_list.append(compute_lbp(img_resized).flatten())
                sift_features_list.append(compute_sift(img_resized).flatten())

                # Store the label
                labels.append(label_folder)

    all_features = np.hstack((np.array(hog_features_list),
                              np.array(lbp_features_list), np.array(cnn_features_list), np.array(sift_features_list)))
    
    scaler = StandardScaler()
    all_features_normalized = scaler.fit_transform(all_features)
    
    return all_features_normalized, np.array(labels)



In [28]:

image_folder = 'dataset/3_dataset/filtered_images'
# Process images and extract features
features_dict, labels = process_and_extract_features_from_folders(image_folder)

print("Feature extraction and normalization completed.")
# print("Features shape:", features_dict.shape)
print("Labels shape:", labels.shape)

1/1 [==============================] - 0s 139ms/step
Feature extraction and normalization completed.
Labels shape: (6157,)


In [29]:
y_training = pd.DataFrame(labels)

In [16]:
from mrmr import mrmr_classif

X_training = pd.DataFrame(features_dict)
y_training = pd.DataFrame(labels)

selected_indices = mrmr_classif(X_training, y_training, K=100, n_jobs=2)
selected = X_training.iloc[:, selected_indices]


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd
import numpy as np

# Assuming you have extracted and selected features using the previous code
# features, labels = process_and_extract_features(image_folder, metadata)
# selected_feature_indices = mrmr_feature_selection(features, labels, num_features_to_select)
# selected_features = features[:, selected_feature_indices]

# Perform train-test split (60% train, 40% test)
X_train, X_test, y_train, y_test = train_test_split(features_dict, y_training, test_size=0.2, random_state=42, stratify=y_training)

# Initialize classifiers
svm_clf = SVC(kernel='linear', random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=3)
/
# Train and evaluate SVM
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)

# Train and evaluate Random Forest
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)

# Train and evaluate KNN
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)




In [31]:
# Evaluation Metrics
def evaluate_model(y_test, y_pred, model_name):
    print(f"Results for {model_name}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print(classification_report(y_test, y_pred, target_names=['Non-Glaucoma', 'Glaucoma']))
    print("------------------------------------------------------")

# Evaluate all models
evaluate_model(y_test, y_pred_svm, "SVM")
evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_knn, "KNN")

Results for SVM:
Accuracy: 0.8003
Precision: 0.7998
Recall: 0.8003
F1-Score: 0.7958
              precision    recall  f1-score   support

Non-Glaucoma       0.80      0.65      0.72       477
    Glaucoma       0.80      0.89      0.85       755

    accuracy                           0.80      1232
   macro avg       0.80      0.77      0.78      1232
weighted avg       0.80      0.80      0.80      1232

------------------------------------------------------
Results for Random Forest:
Accuracy: 0.7955
Precision: 0.8022
Recall: 0.7955
F1-Score: 0.7862
              precision    recall  f1-score   support

Non-Glaucoma       0.84      0.58      0.69       477
    Glaucoma       0.78      0.93      0.85       755

    accuracy                           0.80      1232
   macro avg       0.81      0.76      0.77      1232
weighted avg       0.80      0.80      0.79      1232

------------------------------------------------------
Results for KNN:
Accuracy: 0.6242
Precision: 0.6470
Recall

In [21]:
# # Define the model and feature extraction parameters
# hog_params = {
#     'pixels_per_cell': (8, 8),
#     'cells_per_block': (2, 2),
#     'orientations': 9
# }

# lbp_params = {
#     'radius': 2,
#     'points': 16
# }

# cnn_model = 'Tuned_InceptionResntV2_full_fundus__model.h5'  # or your custom CNN
# input_size = (224, 224, 3)
# dataset_used = "/Users/nemekhbayarnomin/Documents/Graduation_project/dataset/3_dataset/eye_diseases_classification/dataset/half"



In [22]:
# import pandas as pd
# import os

# # File path for the results CSV
# results_file = 'evaluation_results_with_params.csv'

# # Load existing results if the file exists, otherwise create a new DataFrame
# if os.path.exists(results_file):
#     results_df = pd.read_csv(results_file)
# else:
#     results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 
#                                        'HOG Params', 'LBP Params', 'Input Size', 'RGB', 'Dataset', 
#                                        'Feature Extractor', 'mrmr_classif_used'])

# # Define the function to evaluate and save results
# def evaluate_model(y_test, y_pred, model_name, params, df):
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred, average='weighted')
#     recall = recall_score(y_test, y_pred, average='weighted')
#     f1 = f1_score(y_test, y_pred, average='weighted')
    
#     # Append new results to the DataFrame
#     # Append new results to the DataFrame
#     df = df.append({
#         'Model': model_name,
#         'Accuracy': accuracy,
#         'Precision': precision,
#         'Recall': recall,
#         'F1-Score': f1,
#         'HOG Params': str(params['hog_params']),
#         'LBP Params': str(params['lbp_params']),
#         'Input Size': params['input_size'],
#         'RGB': bool(True), 
#         'Dataset': params['dataset_used'],
#         'Feature Extractor': params['cnn_model'],
#         'mrmr_classif_used': bool(False), 
#     }, ignore_index=True)

    
#     print(f"Results for {model_name}:")
#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1-Score: {f1:.4f}")
#     print(classification_report(y_test, y_pred, target_names=['Non-Glaucoma', 'Glaucoma']))
#     print("------------------------------------------------------")
    
#     return df

# # Dictionary to hold parameters for this run
# params = {
#     'hog_params': hog_params,
#     'lbp_params': lbp_params,
#     'input_size': input_size,
#     'dataset_used': dataset_used,
#     'cnn_model': cnn_model
# }

# # Evaluate each model and update the DataFrame with the new results
# results_df = evaluate_model(y_test, y_pred_svm, "SVM", params, results_df)
# results_df = evaluate_model(y_test, y_pred_rf, "Random Forest", params, results_df)
# results_df = evaluate_model(y_test, y_pred_knn, "KNN", params, results_df)

# # Save the updated DataFrame to the CSV file
# results_df.to_csv(results_file, index=False)

# # Display the updated DataFrame
# print(results_df)


Results for SVM:
Accuracy: 0.8750
Precision: 0.8861
Recall: 0.8750
F1-Score: 0.8737
              precision    recall  f1-score   support

Non-Glaucoma       0.95      0.78      0.86       101
    Glaucoma       0.82      0.96      0.89       107

    accuracy                           0.88       208
   macro avg       0.89      0.87      0.87       208
weighted avg       0.89      0.88      0.87       208

------------------------------------------------------
Results for Random Forest:
Accuracy: 0.8894
Precision: 0.8973
Recall: 0.8894
F1-Score: 0.8886
              precision    recall  f1-score   support

Non-Glaucoma       0.95      0.81      0.88       101
    Glaucoma       0.84      0.96      0.90       107

    accuracy                           0.89       208
   macro avg       0.90      0.89      0.89       208
weighted avg       0.90      0.89      0.89       208

------------------------------------------------------
Results for KNN:
Accuracy: 0.6538
Precision: 0.6915
Recall

In [51]:
# results_df.to_csv('cnn-700-rois.csv', index=False)
# results_df